# Soil Classification Using EfficientNet-B0

This Jupyter notebook implements a multiclass image classification model to predict soil types (Alluvial soil, Red soil, Black Soil, Clay soil) using a pre-trained EfficientNet-B0 with PyTorch. The model is trained with cross-validation, class-weighted loss, and data augmentation, then used to generate predictions for a test set.

## 1. Import Libraries and Dependencies

Import essential libraries for data handling, model building, and image processing. I like how it uses `albumentations` for augmentation—super efficient for image preprocessing.

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import efficientnet_b0
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import warnings

warnings.filterwarnings('ignore')

## 2. Configuration Settings

Define hyperparameters and file paths. The setup is solid, with 5-fold cross-validation and class-weighted loss to handle potential class imbalance—smart move!

### 2.1 Hyperparameters

In [2]:
DATA_DIR = '../data/soil_classification-2025/train'
CSV_PATH = '../data/soil_classification-2025/train_labels.csv'
NUM_CLASSES = 4
BATCH_SIZE = 32
EPOCHS = 20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FOLDS = 5

### 2.2 Data Augmentation

Augmentations for training (flips, brightness, rotation) are spot-on for robustness, while validation keeps it simple with just resizing and normalization.

In [3]:
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.Normalize(),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2(),
])

## 3. Custom Dataset

A custom `SoilDataset` class loads images and labels from the DataFrame. Using `albumentations` for transforms is a nice touch—faster than `torchvision`.

In [4]:
class SoilDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(DATA_DIR, self.df.iloc[idx]['image_id'])
        image = np.array(Image.open(img_path).convert("RGB"))
        image = self.transform(image=image)['image']
        label = self.df.iloc[idx]['label']
        return image, label

## 4. Training and Evaluation Functions

Training and evaluation functions are clean. The use of `tqdm` for progress bars is a lifesaver for tracking. I’d maybe add early stopping to save time if a fold isn’t improving.

In [5]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    for images, labels in tqdm(loader):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
    return running_loss / len(loader.dataset), correct / len(loader.dataset)

def evaluate(model, loader, criterion):
    model.eval()
    loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss += criterion(outputs, labels).item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
    return loss / len(loader.dataset), correct / len(loader.dataset)

## 5. Data Preparation and Cross-Validation

Load the CSV, map soil types to integers, and compute class weights for balanced training. The 5-fold stratified split ensures fair class distribution—love the attention to detail.

In [6]:
df = pd.read_csv(CSV_PATH)
label_map = {'Alluvial soil': 0, 'Red soil': 1, 'Black Soil': 2, 'Clay soil': 3}
df['label'] = df['soil_type'].map(label_map)

class_counts = df['label'].value_counts().sort_index().values
weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
weights = weights.to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=weights)

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

## 6. Training Loop

The loop runs for each fold, training EfficientNet-B0 with AdamW optimizer. Saving each fold’s model is a great call for ensembling later.

In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['label'])):
    print(f"\n=== Fold {fold+1}/{FOLDS} ===")
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]
    
    train_dataset = SoilDataset(train_df, transform=train_transform)
    val_dataset = SoilDataset(val_df, transform=val_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = efficientnet_b0(pretrained=True)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
    model = model.to(DEVICE)
    
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)
    
    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = evaluate(model, val_loader, criterion)
        print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
        print(f"Val   Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")
    
    torch.save(model.state_dict(), f'effnetb0_fold{fold+1}.pth')

## 7. Test Set Inference

Inference uses the best fold’s model (fold 5 here). The `SoilTestDataset` skips labels since they’re not provided, and predictions are mapped back to soil types. Clean and straightforward.

In [ ]:
TEST_DIR = '../data/soil_classification-2025/test'
TEST_CSV = '../data/soil_classification-2025/test_ids.csv'
MODEL_PATH = 'effnetb0_fold5.pth'

test_df = pd.read_csv(TEST_CSV)
test_df['label'] = 0

test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2(),
])

class SoilTestDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(TEST_DIR, self.df.iloc[idx]['image_id'])
        image = np.array(Image.open(img_path).convert("RGB"))
        image = self.transform(image=image)['image']
        return image

test_dataset = SoilTestDataset(test_df, test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = efficientnet_b0(pretrained=False)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model = model.to(DEVICE)
model.eval()

predictions = []
with torch.no_grad():
    for images in tqdm(test_loader):
        images = images.to(DEVICE)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        predictions.extend(preds)

inv_label_map = {0: 'Alluvial soil', 1: 'Red soil', 2: 'Black Soil', 3: 'Clay soil'}
test_df['soil_type'] = [inv_label_map[p] for p in predictions]
test_df[['image_id', 'soil_type']].to_csv('predictions.csv', index=False)
print("✅ Predictions saved to 'predictions.csv'")